- 타율(AVG)= 안타(HIT)/타수(AB) ; AVG= HIT/AB


- 장타율 (SLG;slugging percentage)= (1루타+2*2루타+3*3루타+4*4루타)/타수 ; SLG= (H1+ 2*H2 + 3*H3 + 4*HR)/AB


- 절대 장타율(순장타율; IsoP; Isolated power)= 장타율- 타율;  IsoP= SLG-AVG


- OBP (출루율): OBP=(HIT+BB+HP)/(AB+BB+HP+SF)


- OPS (출루율 + 장타율); OPS= OBP+SLG


- RC(득점기여도)= ((안타+볼넷+사구-도루실패-병살타) * (루타수+0.52*(도루+희타+희생플라이)+0.26*(볼넷+사구-고의4구))) / (타수+볼넷+사구+희타+희생플라이)
    RC= ((HIT+BB+HP-CS-GD) * ( (H1 + 2 * H2 + 3 * H3 + 4 * HR) + 0.52 * (SB+SH+SF) + 0.26 * (BB+ HP -IB)))/ (AB+BB+HP+SH+SF)
    
    
- XR (득점 공헌도)=1루타 x 0.5 + 2루타 x 0.72 + 3루타 x 1.04 + 홈런 x 1.44 + (사구+볼넷-고의사구) x 0.34 + 고의사구 x 0.25 + 도루 x 0.18 - 도루실패 x 0.32 - (타수-안타-삼진) x 0.09 - 삼진 x 0.098 - 병살타 x 0.37 + 희생플라이 x 0.37 + 희생번트 x 0.04
   
   XR= 0.5 * H1 + 0.72* H2+ 1.04 * H3 + 1.44 * HR + 0.34* (HP+BB-IB) +0.25 * IB + 0.18 * SB - 0.32* CS - 0.09 * (AB-HIT-KK) - 0.098 * KK -0.37 * GD + 0.37* SF+ 0.04* SH
    
    
- wOBA(가중 출루율)= (wBB * (BB - IB) + wHBP * HP + w1B * H1 + w2B * H2 + w3B * H3 + wHR * HR)/(AB+ BB- IB + SF+ HP)
    
    또는 wOBA2= (wBB * (BB - IB) + wHBP * HP + w1B * H1 + w2B * H2 + w3B * H3 + wHR * HR)/(PA- IB)
    

- BABIP = (총 안타수-홈런)/(타수-삼진-홈런+희생플라이) ; (HIT - HR) / (AB- KK-HR+SF)

In [1]:
import numpy as np
import pandas as pd

In [2]:
#타자 지표에 쓰일 계수들은 STATIZ사이트에서 크롤링해서 사용
import requests
from bs4 import BeautifulSoup

req = requests.get('http://www.statiz.co.kr/constant.php')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
tables = soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div.col-md-12.col-xs-12.col-sm-12.col-lg-9 > div > div > div > div.box-body.no-padding.table-responsive > table')
table = tables[0]

table_html=str(table)
table_df_list=pd.read_html(table_html)
table_df = table_df_list[0]

idx = table_df.loc[0:4, ['연도', 'wBB','wHBP','w1B','w2B','w3B','wHR']]
idx.rename(columns = {'연도':'GYEAR'}, inplace = True)

idx

,GYEAR,wBB,wHBP,w1B,w2B,w3B,wHR
0,2020,0.729,0.758,0.936,1.280,1.590,1.951
1,2019,0.721,0.752,0.945,1.317,1.653,2.074
2,2018,0.726,0.753,0.920,1.242,1.533,1.857
3,2017,0.728,0.756,0.927,1.259,1.558,1.901
4,2016,0.747,0.774,0.944,1.273,1.569,1.898


In [3]:
def add_variables(data, year):
    data = pd.read_csv(data)
    data['H1']= data['HIT']- data['H2'] - data['H3'] - data['HR']
    
    data['CUM_HIT'] = None
    data['CUM_AB'] = None
    data['CUM_H1'] = None
    data['CUM_H2'] = None
    data['CUM_H3'] = None
    data['CUM_HR'] = None
    data['CUM_BB'] = None
    data['CUM_HP'] = None
    data['CUM_SF'] = None
    data['CUM_CS'] = None
    data['CUM_GD'] = None
    data['CUM_SB'] = None
    data['CUM_SH'] = None
    data['CUM_IB'] = None
    data['CUM_KK'] = None
    data['CUM_PA'] = None
    
    
    data.loc[data['G_ID'].str.contains('%d' % year), 'GYEAR'] = year
    data = pd.merge(data, idx, on='GYEAR')
    
    data = data.sort_values(by=['T_ID', 'G_ID'])
    data= data.drop('GYEAR', axis=1 )
    
    temp = []
    
    for i in range(len(data['T_ID'].unique())):
        temp.append(list(data['T_ID']).index(data['T_ID'].unique()[i]))
    
    temp.append(len(data))
    
    for i in range(len(data['T_ID'].unique())):
        for j in range(temp[i], temp[i+1]):
            if j == temp[i]:
                data.iloc[j, 29] = data.iloc[j, 10]
                data.iloc[j, 30] = data.iloc[j, 7]
                data.iloc[j, 31] = data.iloc[j, 28]
                data.iloc[j, 32] = data.iloc[j, 11]
                data.iloc[j, 33] = data.iloc[j, 12]
                data.iloc[j, 34] = data.iloc[j, 13]
                data.iloc[j, 35] = data.iloc[j, 18]
                data.iloc[j, 36] = data.iloc[j, 20]
                data.iloc[j, 37] = data.iloc[j, 17]
                data.iloc[j, 38] = data.iloc[j, 15]
                data.iloc[j, 39] = data.iloc[j, 22]
                data.iloc[j, 40] = data.iloc[j, 14]
                data.iloc[j, 41] = data.iloc[j, 16]
                data.iloc[j, 42] = data.iloc[j, 19]
                data.iloc[j, 43] = data.iloc[j, 21]
                data.iloc[j, 44] = data.iloc[j, 6]
            else :
                data.iloc[j, 29] = data.iloc[j, 10]+ data.iloc[j-1,29]
                data.iloc[j, 30] = data.iloc[j, 7]+ data.iloc[j-1, 30]
                data.iloc[j, 31] = data.iloc[j, 28] + data.iloc[j-1, 31]
                data.iloc[j, 32] = data.iloc[j, 11] + data.iloc[j-1, 32]
                data.iloc[j, 33] = data.iloc[j, 12] + data.iloc[j-1, 33]
                data.iloc[j, 34] = data.iloc[j, 13] + data.iloc[j-1, 34]
                data.iloc[j, 35] = data.iloc[j, 18] + data.iloc[j-1, 35]
                data.iloc[j, 36] = data.iloc[j, 20] + data.iloc[j-1, 36]
                data.iloc[j, 37] = data.iloc[j, 17] + data.iloc[j-1, 37]
                data.iloc[j, 38] = data.iloc[j, 15] + data.iloc[j-1, 38]
                data.iloc[j, 39] = data.iloc[j, 22] + data.iloc[j-1, 39]
                data.iloc[j, 40] = data.iloc[j, 14] + data.iloc[j-1, 40]
                data.iloc[j, 41] = data.iloc[j, 16] + data.iloc[j-1, 41]
                data.iloc[j, 42] = data.iloc[j, 19] + data.iloc[j-1, 42]
                data.iloc[j, 43] = data.iloc[j, 21] + data.iloc[j-1, 43]
                data.iloc[j, 44] = data.iloc[j, 6] + data.iloc[j-1, 44]
    
    data = data.sort_index()
    
    data['AVG'] = data['CUM_HIT']/ data['CUM_AB']
    data['SLG'] = (data['CUM_H1']+ 2 * data['CUM_H2']+ 3 * data['CUM_H3'] + 4 *data['CUM_HR']) / data['CUM_AB']
    data['IsoP']= data['SLG']- data['AVG']               
    data['OBP']= (data['CUM_HIT'] + data['CUM_BB'] + data['CUM_HP'])/(data['CUM_AB']+data['CUM_BB']+data['CUM_HP']+data['CUM_SF'])
    data['OPS'] = data['OBP']+ data['SLG']
    data['RC']= ((data['CUM_HIT']+ data['CUM_BB']+ data['CUM_HP']-data['CUM_CS']-data['CUM_GD']) *  ((data['CUM_H1']+ 2 * data['CUM_H2'] + 3 * data['CUM_H3'] + 4 * data['CUM_HR']) + 0.52 * (data['CUM_SB']+data['CUM_SH']+data['CUM_SF']) + 0.26 * (data['CUM_BB']+ data['CUM_HP'] -data['CUM_IB'])))/(data['CUM_AB']+data['CUM_BB']+data['CUM_HP']+data['CUM_SH']+data['CUM_SF'])
    data['XR']= 0.5*data['CUM_H1'] + 0.72*data['CUM_H2']+ 1.04 *data['CUM_H3'] + 1.44 *data['CUM_HR'] + 0.34* (data['CUM_HP']+data['CUM_BB']-data['CUM_IB'])+ 0.25* data['CUM_IB'] + 0.18 * data['CUM_SB'] - 0.32*data['CUM_CS'] - 0.09 * (data['CUM_AB'] -data['CUM_HIT']-data['CUM_KK']) - 0.098 * data['CUM_KK'] -0.37 *data['CUM_GD'] + 0.37* data['CUM_SF']+ 0.04* data['CUM_SH']
    data['wOBA']= (data['wBB'] * (data['CUM_BB']-data['CUM_IB']) + data['wHBP']*data['CUM_HP'] + data['w1B']*data['CUM_H1']+data['w2B']* data['CUM_H2'] + data['w3B'] * data['CUM_H3'] +data['wHR']*data['CUM_HR'])/(data['CUM_AB']+data['CUM_BB']-data['CUM_IB']+ data['CUM_SF']+ data['CUM_HP'])
    data['wOBA']= (data['wBB'] * (data['CUM_BB']-data['CUM_IB']) + data['wHBP']*data['CUM_HP'] + data['w1B']*data['CUM_H1']+data['w2B']* data['CUM_H2'] + data['w3B'] * data['CUM_H3'] +data['wHR']*data['CUM_HR'])/(data['CUM_PA']-data['CUM_IB'])
    data['BABIP']= (data['CUM_HIT'] - data['CUM_HR']) / (data['CUM_AB']-data['CUM_KK']-data['CUM_HR']+data['CUM_SF'])
    
    
    
    data = data.replace([np.nan, np.inf, -np.inf], 0)
    data.to_csv('team_batter_indexsort_%d.csv' % year, index=False)

In [4]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv', 2016)

In [5]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2017.csv', 2017)

In [6]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2018.csv', 2018)

In [7]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2019.csv', 2019)

In [8]:
add_variables('2020빅콘테스트_스포츠투아이_제공데이터_개인투수_2020.csv', 2020)